# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge


In [26]:
# Load the file
recipes = pd.read_csv("recipes.tsv", sep="\t")

# Inspect data
recipes.head()

,recipe_slug,recipe_title,recipe_introduction
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...


In [27]:
recipes.columns.tolist()

['recipe_slug', 'recipe_title', 'recipe_introduction']

In [28]:
len(recipes)

100

In [29]:
# Copy into a new DataFrame
ratings = recipes[["recipe_slug"]].copy()

# Define rules
def assign_rating(title, intro):
    title_lower = title.lower()
    intro_lower = intro.lower()
    
    # dislikes: bacon
    if "bacon" in title_lower or "bacon" in intro_lower:
        return 0.0
    
    # likes: desserts, sweets
    sweet_keywords = ["cake", "cookie", "brownie", "pie", "ice cream", "pudding",
                      "dessert", "crisp", "candy", "sweet", "chocolate", "muffin"]
    if any(word in title_lower for word in sweet_keywords):
        return 1.0
    
    # medium-high likes: fruits
    fruit_keywords = ["apple", "banana", "peach", "pear", "berry", "lemon"]
    if any(word in title_lower for word in fruit_keywords):
        return 0.75
    
    # medium-low likes: spam, meats
    if "spam" in title_lower or "meat" in title_lower:
        return 0.25
    
    # default: middle ground
    return 0.5

# Apply the rules
ratings["rating"] = recipes.apply(lambda row: assign_rating(row["recipe_title"], row["recipe_introduction"]), axis=1)

# Ensure at least 10 ratings of 0 and 1
n_zeros = (ratings["rating"] == 0).sum()
n_ones = (ratings["rating"] == 1).sum()

# If not enough 0's, force a few extra
if n_zeros < 10:
    extra_idx = ratings[ratings["rating"] == 0.5].sample(10 - n_zeros, random_state=42).index
    ratings.loc[extra_idx, "rating"] = 0.0

# If not enough 1's, force a few extra
if n_ones < 10:
    extra_idx = ratings[ratings["rating"] == 0.5].sample(10 - n_ones, random_state=42).index
    ratings.loc[extra_idx, "rating"] = 1.0

# Save 
output_path = "ratings.tsv"
ratings.to_csv(output_path, sep="\t", index=False)

# Summary counts of ratings
ratings["rating"].value_counts()


rating
0.50    59
1.00    18
0.00    15
0.75     6
0.25     2
Name: count, dtype: int64

In [30]:
ratings.head()

,recipe_slug,rating
0,falafel,0.50
1,spamburger,0.25
2,bacon-fried-rice,0.00
3,chicken-fingers,0.50
4,apple-crisp,1.00


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [31]:
tags = pd.read_csv("recipe-tags.tsv", sep="\t")
tags.head()

,recipe_slug,recipe_tag
0,spam-musubi,hawaiian
1,spam-musubi,nori
2,spam-musubi,onthego
3,spam-musubi,rice
4,spam-musubi,snack


In [32]:
# One-hot encode and solve duplicates with max
onehot = (
    tags.assign(val=1)
        .pivot_table(index="recipe_slug", columns="recipe_tag", values="val",
                     aggfunc="max", fill_value=0)
        .reset_index()
)

onehot.head(10)

recipe_tag,recipe_slug,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,almond-chip-cookies,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,almond-croissants,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,apple-crisp,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,apple-crumble,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,apple-pie,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,asparagus-burger,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,0,0
6,asparagus-quiche,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
7,bacon-and-egg-breakfast-sandwich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,bacon-chocolate-chip-cookies,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,bacon-egg-muffins,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
ratings.head()

,recipe_slug,rating
0,falafel,0.50
1,spamburger,0.25
2,bacon-fried-rice,0.00
3,chicken-fingers,0.50
4,apple-crisp,1.00


In [34]:
# merge
features = ratings[["recipe_slug"]].merge(onehot, on="recipe_slug", how="left").fillna(0)
features

,recipe_slug,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [35]:
tag_cols = [c for c in features.columns if c not in ["recipe_slug"]]
tag_cols

['alfredo',
 'almond',
 'american',
 'appetizer',
 'appetizers',
 'apple',
 'asiancuisine',
 'asparagus',
 'avocado',
 'bacon',
 'baked',
 'bakeddishes',
 'bakery',
 'baking',
 'beans',
 'beef',
 'bellpeppers',
 'berries',
 'blackbeans',
 'blackvinegar',
 'blueberry',
 'boiledeggs',
 'bokchoy',
 'braided',
 'bread',
 'breaded',
 'breakfast',
 'breakfastpastry',
 'brioche',
 'broiling',
 'brownies',
 'brownsugar',
 'brunch',
 'burger',
 'burritos',
 'buttery',
 'cake',
 'carrots',
 'casserole',
 'celebration',
 'cheese',
 'cheesy',
 'cherry',
 'chicken',
 'chickpeas',
 'chilioil',
 'chilipowder',
 'chinese',
 'chinesecookingwine',
 'chinesecuisine',
 'chocolate',
 'chocolatechip',
 'chocolatecroissant',
 'christmas',
 'cilantro',
 'cinnamon',
 'citrus',
 'classic',
 'cloves',
 'cobbler',
 'cocoapowder',
 'coffee',
 'cold',
 'coldnoodles',
 'comfortfood',
 'condiment',
 'cookies',
 'corn',
 'cranberry',
 'creamcheese',
 'creamy',
 'crisp',
 'crispy',
 'croissant',
 'croissants',
 'crumbl

In [36]:
features[tag_cols] = features[tag_cols].astype(int)
features

,recipe_slug,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [37]:
non_meta_cols

['alfredo',
 'almond',
 'american',
 'appetizer',
 'appetizers',
 'apple',
 'asiancuisine',
 'asparagus',
 'avocado',
 'bacon',
 'baked',
 'bakeddishes',
 'bakery',
 'baking',
 'beans',
 'beef',
 'bellpeppers',
 'berries',
 'blackbeans',
 'blackvinegar',
 'blueberry',
 'boiledeggs',
 'bokchoy',
 'braided',
 'bread',
 'breaded',
 'breakfast',
 'breakfastpastry',
 'brioche',
 'broiling',
 'brownies',
 'brownsugar',
 'brunch',
 'burger',
 'burritos',
 'buttery',
 'cake',
 'carrots',
 'casserole',
 'celebration',
 'cheese',
 'cheesy',
 'cherry',
 'chicken',
 'chickpeas',
 'chilioil',
 'chilipowder',
 'chinese',
 'chinesecookingwine',
 'chinesecuisine',
 'chocolate',
 'chocolatechip',
 'chocolatecroissant',
 'christmas',
 'cilantro',
 'cinnamon',
 'citrus',
 'classic',
 'cloves',
 'cobbler',
 'cocoapowder',
 'coffee',
 'cold',
 'coldnoodles',
 'comfortfood',
 'condiment',
 'cookies',
 'corn',
 'cranberry',
 'creamcheese',
 'creamy',
 'crisp',
 'crispy',
 'croissant',
 'croissants',
 'crumbl

In [38]:
# Insert bias
features.insert(1, "bias", 1)

# Sort tag columns 
non_meta_cols = sorted([c for c in features.columns if c not in ["recipe_slug", "bias"]])
features = features[["recipe_slug", "bias"] + non_meta_cols]
features


/tmp/ipykernel_7724/2934080213.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features.insert(1, "bias", 1)


,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [39]:
# Sanity check order matches ratings
assert (features["recipe_slug"].values == ratings["recipe_slug"].values).all(), "Order mismatch with ratings.tsv"


In [40]:
# YOUR CHANGES HERE

# Save
out_path = "features.tsv"
features.to_csv(out_path, sep="\t", index=False)

features.shape, features


((100, 298),
                     recipe_slug  bias  alfredo  almond  american  appetizer  \
 0                       falafel     1        0       0         0          1   
 1                    spamburger     1        0       0         0          0   
 2              bacon-fried-rice     1        0       0         0          0   
 3               chicken-fingers     1        0       0         0          1   
 4                   apple-crisp     1        0       0         0          0   
 ..                          ...   ...      ...     ...       ...        ...   
 95         bacon-mac-and-cheese     1        0       0         0          0   
 96      chicken-alfredo-lasagna     1        1       0         0          0   
 97         classic-beef-lasagna     1        0       0         0          0   
 98  vegetarian-mushroom-lasagna     1        0       0         0          0   
 99  spinach-and-ricotta-lasagna     1        0       0         0          0   
 
     appetizers  apple  a

In [14]:
features.head()

,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [42]:
# YOUR CHANGES HERE

X = features.drop(columns=["recipe_slug"]).values
y = ratings["rating"].values

In [47]:
X

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], shape=(100, 297))

In [46]:
y

array([0.5 , 0.25, 0.  , 0.5 , 1.  , 1.  , 0.  , 0.5 , 0.5 , 0.5 , 1.  ,
       0.  , 0.  , 0.  , 0.5 , 0.5 , 0.  , 0.5 , 1.  , 0.5 , 0.5 , 0.5 ,
       1.  , 1.  , 0.5 , 0.5 , 1.  , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 1.  , 0.5 , 1.  , 0.5 , 0.5 , 0.  , 0.  , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 1.  , 1.  , 1.  , 0.75, 1.  , 0.75, 0.75, 0.5 , 0.75,
       1.  , 0.5 , 1.  , 0.5 , 0.75, 0.25, 0.5 , 0.5 , 0.5 , 0.75, 0.5 ,
       1.  , 0.5 , 0.5 , 0.  , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.  , 1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.5 , 0.5 , 0.5 ,
       0.5 ])

In [43]:
model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X, y)

theta_hat = model.coef_

In [48]:
theta_hat

array([ 3.78460961e-01,  1.50100876e-02,  9.27699398e-02,  7.44271382e-02,
       -1.94439643e-03,  3.69154830e-02,  5.81407881e-02,  1.66564339e-02,
        3.38546215e-03,  2.04977930e-04, -3.55549339e-01,  3.16352763e-03,
       -2.09464941e-03,  5.55939309e-03,  1.36554976e-01,  3.50706527e-03,
        1.46441297e-02,  2.50131499e-03,  6.24961840e-02,  2.50131499e-03,
        3.05500868e-03,  1.05341232e-02,  4.63087213e-03,  4.63087213e-03,
        1.99485174e-02, -1.65708185e-02,  8.01272668e-03, -4.62684540e-02,
       -2.62309062e-03,  4.84788371e-02, -6.25137860e-03,  2.68891111e-02,
        1.11611540e-02,  4.85688574e-02, -3.84348025e-03,  4.47276656e-03,
        6.91644258e-03,  1.55717094e-02,  4.63087213e-03,  1.37829049e-02,
        1.67708487e-03,  3.23287910e-03,  1.08721788e-02, -5.00078109e-02,
        2.56535280e-02, -3.84348025e-03,  3.07101466e-03,  2.50131499e-03,
        1.21503923e-02,  1.60059743e-05,  1.85687893e-02,  1.47373111e-01,
        1.11320392e-01, -

In [44]:
y_hat = model.predict(X)


In [45]:
y_hat

array([ 0.48789551,  0.26369862, -0.0012706 ,  0.47644322,  0.97421494,
        0.98764466,  0.16992904,  0.48943795,  0.49735528,  0.49536913,
        0.96262045,  0.0185091 ,  0.00389692, -0.00447277,  0.48436494,
        0.49444061,  0.02105807,  0.51638613,  0.93110496,  0.54991555,
        0.48425449,  0.4755119 ,  0.97311089,  0.88867961,  0.50103127,
        0.48931467,  0.95152116,  0.48112904,  0.50384348,  0.49998399,
        0.48932367,  0.49832292,  0.49436955,  0.49213377,  0.48201861,
        0.96511153,  0.48668455,  1.00612683,  0.49508582,  0.50372412,
        0.05293924, -0.0069318 ,  0.50209465,  0.49749869,  0.511542  ,
        0.49649293,  0.49593653,  0.4972464 ,  0.48339127,  0.48063744,
        0.49198727,  0.50262309,  0.48801932,  0.50873588,  0.4862171 ,
        0.50741943,  0.48239242,  0.96761819,  0.97880543,  0.99708013,
        0.83490778,  0.9745065 ,  0.69713648,  0.73259671,  0.62443495,
        0.75119418,  0.89723753,  0.50039329,  0.98005148,  0.57

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [ ]:
# YOUR CHANGES HERE

...

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE

...

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [ ]:
# YOUR CHANGES HERE

...

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.